#### AI and Bond Values: How Large Language Models Predict Default Signals

Setting Stage

In [50]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load

In [9]:
environment = 'PROD'
proj_folder = '//LSFS/BOSTON/Dept/FIM/HF2/ChatGPT/LLM2025'
data_folder = '//LSFS/BOSTON/Dept/FIM/HF2/Systematic Credit Process/LLM Project/data/416472'
#os.chdir(projFolder)

In [10]:
entity_id = "590652"
start_date = "01/01/2024" #format: mm/dd/yyyy
end_date = "01/01/2025" #format: mm/dd/yyyy

In [ ]:
#df_rpt: timeseries of all earnings reports by securityId
from FSPortApiData import get_reports, load_reports
get_reports(entity_id, start_date, end_date, proj_folder)
df_sec, df_rpt = load_reports(entity_id, start_date, end_date, proj_folder)

...downloading earnings report......


In [ ]:
#example how to retrieve the reports of selected security
df_rpt[df_rpt["securityId"] == 67085602]

,securityId,report_id,factset_entity_id,event_type,event_datetime_utc,overall_default_score,overall_default_score_comment,financial_performance_risk,financial_performance_risk_comment,liquidity_debt_risk,liquidity_debt_risk_comment,operational_challenge_risk,operational_challenge_risk_comment,market_competitive_risk,market_competitive_risk_comment,legal_regulation_risk,legal_regulation_risk_comment,broad_economic_factor_risk,broad_economic_factor_risk_comment
217,67085602,3110176,0088V7-E,E,2024-10-31T00:30:00.000+00:00,6,While facing some operational challenges and a...,5,The company reported a consolidated net loss b...,4,"Liquidity is strong with $1.1 billion, debt is...",7,Significant disruptions from hurricanes and op...,5,"The airline faces competitive pressure, yet it...",3,No significant legal or regulatory concerns we...,6,Fluctuating fuel prices and economic condition...
218,67085602,2976017,0088V7-E,E,2024-02-06T01:00:00.000+00:00,4,The company has shown strong operational metri...,4,Adjusted net income is positive with a signifi...,5,The company has substantial liquidity and has ...,5,Navigating pilot labor issues and uncertainty ...,6,Increased competition from major carriers in t...,5,Pending regulatory approvals for international...,6,Current economic conditions and fuel prices ca...
219,67085602,3068292,0088V7-E,E,2024-08-01T00:30:00.000+00:00,6,"The company faces several challenges, includin...",5,Financial performance shows mixed results with...,4,Liquidity appears manageable with a solid cash...,6,Operational challenges are significant due to ...,7,The competitive landscape is tough as Allegian...,5,Regulatory changes and potential litigation pr...,6,"Economic factors, such as rising fuel costs an..."
220,67085602,3035730,0088V7-E,E,2024-05-07T20:30:00.000+00:00,6,The company faces several operational and fina...,6,"The first quarter results were disappointing, ...",5,The company has substantial liquidity of $1.1 ...,7,The company is experiencing significant operat...,5,The company operates in a competitive environm...,4,There are new regulatory challenges but the co...,6,The broader economic environment remains unpre...


In [44]:
ts_cols = ['date', 'rep_sec_id', 'bench_wt', 'market_value', 
           'oas', 'oad', 'duration', 'oasd', 'yrs_to_mat', 
           'krd_6m', 'krd_2y', 'krd_5y', 'krd_10y', 'krd_20y', 'krd_30y', 
           'convexity', 'yield_to_worst', 'yield_to_mat', 'amt_out', 
           'price', 'acc_int','yrs_to_avglife', 'currency_code', 'trace_vol_20bd', 
           'total_return_1dfwd', 'excess_return_1dfwd', 'currency_return_1dfwd', 
           'TotalDebt_MarketCap', 'TotalDebt_TotalAssets', 'Q10_ROA', 'EBIT_TTM_Interest_TTM', 
           'Q2_OperatingMargin', 'Q3_NetincomeMargin', 'Q1_GrossMargin', 'Q9_ROE', 
           'TotalDebt', 'CFO_TotalDebt', 'Q8_AssetTO', 'MarketCap_Quarterly_', 
           'Q6_CurrentRatio', 'EBITDA_TTM_Margin', 'V1_TrailingEarningsYield', 
           'CFO_CurrentLiabilities', 'FCF_TTM_CFO_TTM']

In [ ]:
#df_data: mapped security master with mapping of securityId to rep_sec_id
str_date = "20240102"
df_data = pd.read_excel(data_folder+'/416472_'+str_date+'.xlsx')
df_data = df_data[df_data["rep_sec_id"].isin(df_rpt["securityId"].to_list())]
df_data = df_data.merge(df_sec, left_on="rep_sec_id", right_on="securityId", suffixes=('', '_'))
df_data = df_data[list(set(df_data.columns)-set(ts_cols[2:]))]

In [ ]:
#df_ts: timeseries of all analytics by rep_sec_id
dt_date = datetime.datetime.strptime(start_date, "%m/%d/%Y")
dt_end = datetime.datetime.strptime(end_date, "%m/%d/%Y")
str_date = dt_date.strftime("%Y%m%d")
df_ts = []
while dt_date < dt_end:
    try:
        str_date = dt_date.strftime("%Y%m%d")
        tmp_data = pd.read_excel(data_folder+'/416472_'+str_date+'.xlsx')
        tmp_data = tmp_data[ts_cols]
        tmp_data = tmp_data[tmp_data["rep_sec_id"].isin(df_data["rep_sec_id"].to_list())]
        df_ts.append(tmp_data)
    except:
        pass
    dt_date = dt_date + datetime.timedelta(days=1)
df_ts = pd.concat(df_ts)

In [ ]:
#example how to retrieve the timeseries of selected security
df_ts[df_ts["rep_sec_id"] == 67085602]

,date,rep_sec_id,bench_wt,market_value,oas,oad,duration,oasd,yrs_to_mat,krd_6m,...,Q9_ROE,TotalDebt,CFO_TotalDebt,Q8_AssetTO,MarketCap_Quarterly_,Q6_CurrentRatio,EBITDA_TTM_Margin,V1_TrailingEarningsYield,CFO_CurrentLiabilities,FCF_TTM_CFO_TTM
66,2024-01-02,67085602,0.000440,552625.9987,364.7331,2.5397,2.5397,2.5454,3.6137,0.068350,...,0.132105,3569.191,0.126271,0.530360,1249.719908,NaN,0.213088,0.136782,0.429349,-0.800078
949,2024-01-03,67085602,0.000442,552448.7492,380.2903,2.5412,2.5412,2.5640,3.6110,0.069341,...,0.132105,3569.191,0.126271,0.530360,1507.364018,NaN,0.213088,0.113402,0.429349,-0.800078
932,2024-01-04,67085602,0.000438,545874.1856,407.7371,2.6131,2.6131,2.6356,3.6082,0.053083,...,0.132105,3569.191,0.126271,0.530360,1507.364018,NaN,0.213088,0.113402,0.429349,-0.800078
932,2024-01-05,67085602,0.000437,544959.9787,418.6698,2.6316,2.6316,2.6637,3.6055,0.046686,...,0.132105,3569.191,0.126271,0.530360,1507.364018,NaN,0.213088,0.113402,0.429349,-0.800078
932,2024-01-08,67085602,0.000439,548286.7935,396.5159,2.5769,2.5769,2.5972,3.5973,0.061386,...,0.132105,3569.191,0.126271,0.530360,1507.364018,NaN,0.213088,0.113402,0.429349,-0.800078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,2024-12-23,67085602,0.000420,564903.1513,246.4218,1.8039,1.8039,1.7934,2.6411,0.182330,...,-0.019656,3488.352,0.088211,0.512071,1498.306260,NaN,0.134323,-0.017330,0.249247,-0.676683
990,2024-12-24,67085602,0.000420,565013.9151,245.9355,1.8025,1.8025,1.7919,2.6384,0.179980,...,-0.019656,3488.352,0.088211,0.512071,1498.306260,NaN,0.134323,-0.017330,0.249247,-0.676683
970,2024-12-26,67085602,0.000420,565594.6843,248.4508,1.7985,1.7985,1.7898,2.6329,0.181910,...,-0.019656,3488.352,0.088211,0.512071,1498.306260,NaN,0.134323,-0.017330,0.249247,-0.676683
971,2024-12-27,67085602,0.000420,566128.1247,240.1064,1.7777,1.7777,1.7695,2.6301,0.194300,...,-0.019656,3488.352,0.088211,0.512071,1498.306260,NaN,0.134323,-0.017330,0.249247,-0.676683
